In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [ ]:
# Paths to input datasets
tc_ds = r"Z:\PhD_Datasets&Analysis\Info_Inputs\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"
bands_gee = ["pr", "pet", "ro"] # band names in gee
tc_vars = ["ppt", "pet", "q"] # variable names according to TerraClimate
years = range(1958, 2023 + 1) # Years with available weather information to run the water balance
months = range(1, 12 + 1)

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")

In [ ]:
print('\n############################################################')
print('\t\tINITIAL VARIABLES')
print('\tPeriod to be executed: ' + str(years[0]) + '-' + str(years[-1]))
print('############################################################')

for year in years:

    print("\n**Executing conversfion for " + str(year) + "**")

    ppt_nc_file = tc_ds + "\\TerraClimate_ppt_" + str(year) + ".nc"
    pet_nc_file  = tc_ds + "\\TerraClimate_pet_" + str(year) + ".nc"
    q_nc_file = tc_ds + "\\TerraClimate_q_" + str(year) + ".nc"

    nc_FP = arcpy.NetCDFFileProperties(ppt_nc_file)

    for month in months:

        print("\n\t*Executing conversion for month " + str(month) + "*\n")

        print("\tLoading precipitation, potential evapo., and runoff rasters from TerraClimate NetCDF files.....")

        dimension_value = nc_FP.getDimensionValue("time", month-1)

        pr_name = "ppt_" + str(year) + "_" + str(month)
        pet_name = "pet_" + str(year) + "_" + str(month)
        ro_name = "q_" + str(year) + "_" + str(month)

        arcpy.MakeNetCDFRasterLayer_md(ppt_nc_file, "ppt", "lon", "lat", pr_name, "", [["time", dimension_value]], "BY_VALUE")
        arcpy.MakeNetCDFRasterLayer_md(pet_nc_file, "pet", "lon", "lat", pet_name, "", [["time", dimension_value]], "BY_VALUE")
        arcpy.MakeNetCDFRasterLayer_md(q_nc_file, "q", "lon", "lat", ro_name, "", [["time", dimension_value]], "BY_VALUE")

        print("\tConverting the files 'ppt', 'pet', and 'ro'.....")
        # Save the raster as GeoTIFF
        arcpy.CopyRaster_management(pr_name, out_geotiff + "\\" + pr_name + ".tif", format="TIFF")
        arcpy.CopyRaster_management(pet_name, out_geotiff + "\\" + pet_name + ".tif", format="TIFF")
        arcpy.CopyRaster_management(ro_name, out_geotiff + "\\" + ro_name + ".tif", format="TIFF")

arcpy.CheckInExtension("spatial")

# Clear the workspace environment
arcpy.ClearEnvironment("workspace")

print("\nDONE!!")